# Notebook for classifying claims and non-claims with BiLSTM and CNN

In [1]:
from scripts.load_corpus import DaxenbergerModified, StabGurevychCorpus

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D


Loading corpus from script class

In [2]:
#corpus_1 = StabGurevychCorpus()
#df_all = corpus_1.df_all
#print(df_all.head)

corpus_2 = DaxenbergerModified()
df_all = corpus_2.df_all
print(df_all.head)

<bound method NDFrame.head of                                                    text  target
0     How can anyone expect children could do well a...       0
1     Firstly , I think that the new high school wil...       1
2     With technological advances , children have mo...       0
3     Nowadays , many professors conduct research wh...       0
4     In today ' s world there are many great and us...       0
...                                                 ...     ...
7046  Last but not least , knowledge is worth mentio...       1
7047  To illustrate this point , I can write about m...       0
7048  Consider a circumstance in which a student who...       0
7049  in my opinion , reducing stress by listening t...       1
7050  In addition , the basic economic course can al...       1

[7051 rows x 2 columns]>


Hyperparameters used in BiLSTM and CNN models

In [3]:
vocab_size = 25000
embedding_dim = 300
input_n = 50 # padded maximum length for each sample

Encoding words from corpus

In [4]:
encoding = [one_hot(words,vocab_size) for words in df_all.iloc[:, 0]]

Padding to bring all the sequences to the same length

In [5]:
emb_doc = pad_sequences(encoding, padding='pre', maxlen=input_n)
print(emb_doc)

[[    0     0     0 ... 13661  3001 23758]
 [    0     0     0 ... 19048  9096   907]
 [    0     0     0 ...  8435 10505  4132]
 ...
 [    0     0     0 ...  9096  7101 22116]
 [    0     0     0 ...  9581  9392 11974]
 [    0     0     0 ...  8178  4951 21704]]


Converting data back to arrays and splitting it into train, validation, and 
test set

In [6]:
X = np.array(emb_doc)
y = np.array(df_all.iloc[:, 1])

random_seeds = [0, 1, 2, 3, 4]

X_train, X_rem, y_train, y_rem = train_test_split(X, y,
                                                  train_size=0.7,
                                                  random_state=random_seeds[4])
print(X_train.shape)

X_tr = np.column_stack((X_train, y_train))

positive_entries = X_tr[X_tr[:, -1] == 1]
negative_entries = X_tr[X_tr[:, -1] == 0]

min_size = min(len(positive_entries), len(negative_entries))

if len(positive_entries) > len(negative_entries):
    positive_entries = positive_entries[:min_size]
else:
    negative_entries = negative_entries[:min_size]

X_tr = np.concatenate((positive_entries, negative_entries))
np.random.shuffle(X_tr)
X_train = X_tr[:, :50]
y_train = X_tr[:,-1]

print(X_train.shape)
print(y_train.shape)

print(X_rem.shape)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, 
                                                    test_size=0.5,
                                                    random_state=42)
print(X_valid.shape)
print(X_test.shape)

(4935, 50)
(2924, 50)
(2924,)
(2116, 50)
(1058, 50)
(1058, 50)


## BiLSTM

Defining BiLSTM model

In [7]:
bilstm_model = Sequential()
bilstm_model.add(Embedding(vocab_size, embedding_dim, input_length=input_n)) 
bilstm_model.add(Bidirectional(LSTM(100)))
bilstm_model.add(Dense(1, activation='sigmoid'))
bilstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(bilstm_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 300)           7500000   
                                                                 
 bidirectional (Bidirection  (None, 200)               320800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 7821001 (29.83 MB)
Trainable params: 7821001 (29.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Training the Bi-LSTM model

In [8]:
bilstm_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=512)

Epoch 1/10
6/6 [==============================] - 3s 358ms/step - loss: 0.6909 - accuracy: 0.5250 - val_loss: 0.6964 - val_accuracy: 0.4565
Epoch 2/10
6/6 [==============================] - 2s 278ms/step - loss: 0.6701 - accuracy: 0.5947 - val_loss: 0.7065 - val_accuracy: 0.4877
Epoch 3/10
6/6 [==============================] - 2s 281ms/step - loss: 0.6395 - accuracy: 0.6310 - val_loss: 0.6548 - val_accuracy: 0.5851
Epoch 4/10
6/6 [==============================] - 2s 284ms/step - loss: 0.5800 - accuracy: 0.7158 - val_loss: 0.7456 - val_accuracy: 0.4924
Epoch 5/10
6/6 [==============================] - 2s 264ms/step - loss: 0.5034 - accuracy: 0.7654 - val_loss: 0.6728 - val_accuracy: 0.6163
Epoch 6/10
6/6 [==============================] - 2s 268ms/step - loss: 0.3921 - accuracy: 0.8505 - val_loss: 0.6595 - val_accuracy: 0.6767
Epoch 7/10
6/6 [==============================] - 2s 276ms/step - loss: 0.3098 - accuracy: 0.8796 - val_loss: 0.7382 - val_accuracy: 0.6664
Epoch 8/10
6/6 [====

Evaluation with validation set:

Prediction results:
1 = claim
0 = non-claim

In [9]:
y_pred = [value for value in (bilstm_model.predict(X_valid) > 0.5).astype("int32")]
cr = metrics.classification_report(y_valid.tolist(), y_pred, digits=4)
print(cr)

34/34 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0     0.8338    0.7360    0.7819       750
           1     0.5000    0.6429    0.5625       308

    accuracy                         0.7089      1058
   macro avg     0.6669    0.6894    0.6722      1058
weighted avg     0.7367    0.7089    0.7180      1058



Evaluation with test set:

Prediction results:
1 = claim
0 = non-claim

In [10]:
y_pred_test = [value for value in (bilstm_model.predict(X_test) > 0.5).astype("int32")]
cr = metrics.classification_report(y_test.tolist(), y_pred_test, digits=4)
print(cr)

34/34 [==============================] - 0s 13ms/step
              precision    recall  f1-score   support

           0     0.8211    0.7429    0.7800       735
           1     0.5191    0.6316    0.5698       323

    accuracy                         0.7089      1058
   macro avg     0.6701    0.6872    0.6749      1058
weighted avg     0.7289    0.7089    0.7158      1058



## CNN

Defining CNN model

In [11]:
cnn_model = Sequential()
cnn_model.add(Embedding(vocab_size, embedding_dim, input_length=input_n))
cnn_model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 300)           7500000   
                                                                 
 conv1d (Conv1D)             (None, 50, 128)           153728    
                                                                 
 max_pooling1d (MaxPooling1  (None, 25, 128)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 64)            32832     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 12, 32)           

Training the CNN model

In [12]:
cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=10, 
              batch_size=512, verbose=1)

Epoch 1/10
6/6 [==============================] - 1s 127ms/step - loss: 0.6907 - accuracy: 0.5466 - val_loss: 0.6756 - val_accuracy: 0.6109
Epoch 2/10
6/6 [==============================] - 1s 111ms/step - loss: 0.6682 - accuracy: 0.5952 - val_loss: 0.6392 - val_accuracy: 0.6314
Epoch 3/10
6/6 [==============================] - 1s 113ms/step - loss: 0.6044 - accuracy: 0.6864 - val_loss: 0.5763 - val_accuracy: 0.7099
Epoch 4/10
6/6 [==============================] - 1s 113ms/step - loss: 0.4669 - accuracy: 0.8084 - val_loss: 0.5631 - val_accuracy: 0.7167
Epoch 5/10
6/6 [==============================] - 1s 113ms/step - loss: 0.3096 - accuracy: 0.8784 - val_loss: 0.6521 - val_accuracy: 0.6451
Epoch 6/10
6/6 [==============================] - 1s 116ms/step - loss: 0.1742 - accuracy: 0.9396 - val_loss: 0.8071 - val_accuracy: 0.6860
Epoch 7/10
6/6 [==============================] - 1s 114ms/step - loss: 0.1082 - accuracy: 0.9643 - val_loss: 0.9142 - val_accuracy: 0.6451
Epoch 8/10
6/6 [====

Evaluation with validation set:

Prediction results: \
1 = claim \
0 = non-claim

In [13]:
y_pred = [value for value in (cnn_model.predict(X_valid) > 0.5).astype("int32")]
cr = metrics.classification_report(y_valid.tolist(), y_pred, digits=4)
print(cr)

34/34 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0     0.8580    0.5720    0.6864       750
           1     0.4247    0.7695    0.5473       308

    accuracy                         0.6295      1058
   macro avg     0.6414    0.6707    0.6169      1058
weighted avg     0.7319    0.6295    0.6459      1058



Evaluation with test set:

Prediction results: \
1 = claim \
0 = non-claim 

In [14]:
y_pred_test = [value for value in (cnn_model.predict(X_test) > 0.5).astype("int32")]
cr = metrics.classification_report(y_test.tolist(), y_pred_test, digits=4)
print(cr)

34/34 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0     0.8356    0.5878    0.6901       735
           1     0.4399    0.7368    0.5509       323

    accuracy                         0.6333      1058
   macro avg     0.6378    0.6623    0.6205      1058
weighted avg     0.7148    0.6333    0.6476      1058



corpus 1 \
model , macro F1, claim F1

1st iteration \
BiLSTM: 0.6534 0.5829 \
CNN: 0.6122; 0.5393

2nd iteration \
BiLSTM: 0.6405; 0.5816 \
CNN: 0.6184; 0.5420

3rd iteration \
BiLSTM: 0.6417; 0.5605 \
CNN: 0.6061; 0.5593

4th iteration \
BiLSTM: 0.6388; 0.5749 \
CNN: 0.6313; 0.5737

5th iteration \
BiLSTM: 0.6446, 0.6098 \
CNN: 0.5876; 0.5586

In [15]:
# average results for corpus 1

bilstm_mean_macro_f1 = (0.6534 + 0.6405 + 0.6417 + 0.6388 + 0.6446) / 5
bilstm_mean_claim_f1 = (0.5829 + 0.5816 + 0.5605 + 0.5749 + 0.6098) / 5

cnn_mean_macro_f1 = (0.6122 + 0.6184 + 0.6061 + 0.6313 + 0.5876) / 5
cnn_mean_claim_f1 = (0.5393 + 0.5420 + 0.5593 + 0.5737 + 0.5586) / 5

print("BiLSTM")
print(bilstm_mean_macro_f1)
print(bilstm_mean_claim_f1)
print("CNN")
print(cnn_mean_macro_f1)
print(cnn_mean_claim_f1)

BiLSTM
0.6437999999999999
0.58194
CNN
0.61112
0.55458


corpus 2 \
model , macro F1, claim F1

1st iteration \
BiLSTM: 0.6638; 0.5911 \
CNN: 0.6430; 0.5803

2nd iteration \
BiLSTM: 0.6470; 0.5687 \
CNN: 0.5975; 0.5545

3rd iteration \
BiLSTM 0.6223; 0.5345 \
CNN: 0.6263; 0.5316

4th iteration \
BiLSTM: 0.6311; 0.5533 \
CNN: 0.6476; 0.5530

5th iteration \
BiLSTM: 0.6568; 0.5413 \
CNN: 0.6314; 0.5569

In [16]:
# average results for corpus 2

bilstm_mean_macro_f1 = (0.6638 + 0.6470 + 0.6223 + 0.6311 + 0.6568) / 5
bilstm_mean_claim_f1 = (0.5911 + 0.5687 + 0.5345 + 0.5533 + 0.5413) / 5

cnn_mean_macro_f1 = (0.6430 + 0.5975 + 0.6263 + 0.6476 + 0.6314) / 5
cnn_mean_claim_f1 = (0.5803 + 0.5545 + 0.5316 + 0.5530 + 0.5569) / 5

print("BiLSTM")
print(bilstm_mean_macro_f1)
print(bilstm_mean_claim_f1)
print("CNN")
print(cnn_mean_macro_f1)
print(cnn_mean_claim_f1) 

BiLSTM
0.6442
0.5577799999999999
CNN
0.62916
0.55526
